# 1. Phương pháp phân tích suy biến.

## 1.1. Giới thiệu chung.
Phương pháp *phân tích suy biến* (singular value decomposition) được viết tắt là SVD là một trong những phương pháp thuộc nhóm matrix factorization được phát triển lần đầu bởi những nhà hình học vi phân. Ban đầu mục đích của phương pháp này là tìm ra một phép xoay không gian sao cho tích vô hướng của các vector không thay đổi. Từ mối liên hệ này khái niệm về ma trận trực giao đã hình thành để tạo ra các phép xoay đặc biệt. Phương pháp SVD đã được phát triển dựa trên những tính chất của ma trận trực giao và ma trận đường chéo để tìm ra một ma trận xấp xỉ với ma trận gốc. Phương pháp này sau đó đã được ứng dụng rộng rãi trong các lĩnh vực như hình học vi phân, hồi qui tuyến tính, xử lý hình ảnh, clustering, các thuật toán nèn và giảm chiều dữ liệu, và đặc biệt đặc biệt hiệu quả trong các bài toán recommendation mà ta sẽ trình bày ở cuối bài viết.

### 1.1.1. Mục tiêu của phân tích suy biến.
Phương pháp SVD sẽ tìm ra một lớp các ma trận xấp xỉ tốt nhất với một ma trận cho trước dựa trên khoảng cách norm Frobenios giữa 2 ma trận. Người ta đã chứng minh được rằng ma trận xấp xỉ tốt nhất được biểu diễn dưới dạng tích của 3 ma trận rất đặc biệt bao gồm 2 *ma trận trực giao* (orthogonal matrix) và 1 *ma trận đường chéo* (diagonal matrix). Quá trình nhân ma trận thực chất là quá trình biến đổi các điểm dữ liệu của ma trận gốc thông qua những phép xoay trục (rotation) và phép thay đổi độ lớn (scaling) và từ đó tạo ra những điểm dữ liệu mới trong không gian mới. Điều đặc biệt của ma trận đường chéo đó là các phần tử của nó chính là những giá trị riêng của ma trận gốc. Những điểm dữ liệu trong không gian mới có thể giữ được 100% thông tin ban đầu hoặc chỉ giữ một phần lớn thông tin của dữ liệu ban đầu thông qua các phép truncate SVD. Bằng cách sắp xếp các trị riêng theo thứ tự giảm dần trên đường chéo chính thuật toán SVD có thể thu được ma trận xấp xỉ tốt nhất mà vẫn đảm bảo giảm được hạng của ma trận sau biến đổi và kích thước các ma trận nhân tử nằm trong giới hạn cho phép. Do đó nó tiết kiệm được thời gian và chi phí tính toán và đồng thời cũng tìm ra được một giá trị dự báo cho ma trận gốc với mức độ chính xác cao. 

Trên đây là những thông tin chung nhất về thuật toán SVD. Chắc rằng chúng ta sẽ rất mơ hồ khi mới lần đầu tiếp cận phương pháp này. Chính vì vậy kiến thức về đại số tuyến tính sẽ được hệ thống trước khi đi vào thuật toán.

## 1.2. Kiến thức chung về đại số tuyến tính.
**Hệ trực giao:** Trong đại số tuyến tính, hệ trực giao là một lý thuyết quan trọng. Một hệ vector cơ sở $\mathbf{U = [u_1, u_2, ..., u_n]} \in \mathbb{R}^{n}$ được gọi là một *hệ trực giao* (orthogonal) nếu thỏa mãn hệ điều kiện:
$$\mathbf{||u_i||_{2}^{2}} > 0, \mathbf{u_{i}^{T}u_{j}} = 0 ~ \forall 1 \leq i \neq j \leq n$$

**Hệ trực chuẩn:** *Hệ trực chuẩn* (orthonormal) là một trường hợp đặc biệt của hệ trực giao khi điều kiện $\mathbf{||u_i||_{2}^{2}} > 0$ được thay thể thành $\mathbf{||u_i||_{2}^{2}} = 1$. 

**Ma trận trực giao:** *Ma trận trực giao* (orthogonal matrix) là ma trận vuông thỏa mãn các dòng và cột của nó là một hệ trực chuẩn. Điều đó có nghĩa là một ma trận $\mathbf{U} \in \mathbb{R}^{n \times n}$ sẽ bao gồm các cột $\mathbf{u}_{i} in \mathbb{R}^{n}$ có tính chất:
$$\mathbf{||u_i||_{2}^{2}} > 0, \mathbf{u_{i}^{T}u_{j}} = 0 ~ \forall 1 \leq i \neq j \leq n$$
Từ đó suy ra 
$$\mathbf{U^TU = I_{n}}$$

Ma trận trực giao có những tính chất đặc biệt sau:
1. Nếu $\mathbf{U}$ trực giao thì $\mathbf{U}^{T}$ cũng trực giao.
2. Ma trận trực giao có chuyển vị bằng ma trận nghịch đảo. Từ đẳng thức $\mathbf{U^TU = I_{n}}$ suy ra $\mathbf{U^{T} = U^{-1}}$.
3. Nếu ma trận $\mathbf{U} \in \mathbb{R}^{n \times n}$ là ma trận trực giao thì ma trận $\mathbf{U_r} \in \mathbb{R}^{n \times r}$ được rút ra từ $r$ cột đầu tiên của ma trận $\mathbf{U}$ có tính chất: $\mathbf{U_r^{T}U_r = I_r}$.
Để chứng minh tính chất này ta cần sử dụng đến khai triển tích của 2 ma trận dựa trên các ma trận con (được trích ra từ các dòng và cột của ma trận gốc). Giả sử $\mathbf{A ,B} \in \mathbb{R}^{n \times n}$. Ma trận $\mathbf{M} \in \mathbb{R}^{r \times n}$ gồm $r$ dòng đầu tiên của ma trận $\mathbf{A}$ và ma trận $\mathbf{\bar{M}}$ là ma trận phần bù của $\mathbf{U}$ được tạo thành từ các dòng còn lại. Tương tự, ma trận $\mathbf{N} \in \mathbb{R}^{n \times r}$ gồm $r$ cột đầu tiên của ma trận $\mathbf{B}$ và $\mathbf{\bar{N}}$ là ma trận phần bù của $\mathbf{V}$. Khi đó tích $\mathbf{AB}$ được khai triển theo $\mathbf{M,N}$ và các ma trận phần bù như sau:
$$\begin{bmatrix}\mathbf{AB}\end{bmatrix} = 
\begin{bmatrix}\mathbf{M} \\ \mathbf{\bar{M}}\end{bmatrix}
\begin{bmatrix}\mathbf{N} ~~ \mathbf{\bar{N}}\end{bmatrix} = 
\begin{bmatrix}\mathbf{MN} ~~ \mathbf{M\bar{N}}\\ \mathbf{\bar{M}N} ~~ \mathbf{\bar{M}\bar{N}}\end{bmatrix}
$$
Khi áp dụng tính chất trên cho ma trận $\mathbf{U_r}$ ta suy ra tích $\mathbf{U_r^{T}U_r}$ sẽ là ma trận được tạo thành bởi r dòng và r cột đầu tiên của tích $\mathbf{U^TU}$. Do đó $\mathbf{U_r^{T}U_r}= \mathbf{I_r}$.
4. Ma trận trực giao không làm thay đổi giá trị tích vô hướng của 2 vector. Thật vậy, nếu coi $\mathbf{x, y} \in \mathbb{R}^{n}$ là các vector được biến đổi bằng các nhân với ma trận trực giao $\mathbf{U} \in \mathbb{R}^{n \times n}$. Khi đó tích vô hướng của 2 vector mới như sau:
$$\mathbf{(Ux)^{T}Uy} = \mathbf{x^{T}U^{T}Uy} = \mathbf{x^Ty}$$

**Ma trận đường chéo:** Một ma trận $\mathbf{D}$ là ma trận đường chéo khi các phần tử của nó thỏa mãn:
$$d_{ii} \neq 0, d_{ij} = 0 ~ \forall i \neq j$$
Hay nói cách khác ma trận có các phần tử trên đường chéo chính khác 0 và các phần tử còn lại bằng 0. Ma trận đơn vị $\mathbf{I_n}$ là một trường hợp đặc biệt của ma trận đường chéo. Ma trận đường chéo có thể không vuông. Ngoài ra ta có thể nhận thấy mối liên hệ giữa hệ trực giao và ma trận đường chéo đó là một ma trận $\mathbf{U} \in \mathbb{R}^{n \times n}$ có các cột tạo thành một hệ trực giao thì tích của nó với ma trận chuyển vị sẽ tạo thành một ma trận đường chéo.

**Trace của ma trận:** hàm *trace* của ma trận $\mathbf{A}$ kí hiệu là $\mathbf{trace(A)}$ là tổng của tất cả các phần tử trên đường chéo của ma trận đó. 
Một số đẳng thức của *trace*:

1. $\mathbf{trace(A) = trace(A^{T})}$. Điều này là hiển nhiên do phép chuyển vị không làm thay đổi các vị trí trên đường chéo chính của $\mathbf{A}$.

2. $\mathbf{trace(AB) = trace(BA)}$. Chứng minh tính chất này khá đơn giản. Giả sử $\mathbf{A}^{(i)}$ là vector dòng thứ i của $\mathbf{A}$ và $\mathbf{B}_{j}$ là vector cột thứ j của $\mathbf{B}$. Khi đó phần tử $AB_{ij}$ ở dòng thứ i cột thứ j của ma trận tích $\mathbf{AB}$ là:
$$AB_{ij} = \mathbf{A}^{(i)}\mathbf{B}_{j} = \sum_{k}(a_{ik}b_{kj})$$
Do đó:
$$\mathbf{trace(AB)} = \sum_{i} AB_{ii} = \sum_{i}\sum_{k}(a_{ik}b_{ki}) \tag{1}$$
Hoàn toàn biến đổi tương tự:
$$\mathbf{trace(BA)} = \sum_{i} BA_{ii} = \sum_{i}\sum_{k}(b_{ik}a_{ki}) \tag{2}$$
Ta nhận thấy chỉ số $i, k$ bình đẳng trong cả 2 biểu thức (1) và (2) nên nếu hoán vị i và k cho nhau không làm thay đổi kết quả tổng. Mặt khác phép hoán vị này sẽ biến biểu thức (1) thành (2) nên suy ra giá trị của 2 biểu thức là bằng nhau.
3. $\mathbf{trace(ABC) = trace(CAB) = trace(BCA)}$. Tính chất này suy ra từ tính chất 2.
4. $\mathbf{trace(A + B) = trace(A) + trace(B)}$. Dễ dàng suy ra tính chất này do $\mathbf{A, B}$ phải cùng kích thước và vị trí các phần tử trên đường chéo chính của 2 ma trận này là trùng nhau.

**Trị riêng của ma trận:** Chúng ta chắc hẳn đã làm quen rất nhiều với trị riêng (eigen-value) của một ma trận vuông. Đó là một đại lượng vô hướng (scaler) có tính chất đặc biệt sau: ma trận $\mathbf{A} \in \mathbb{R}^{n \times n}$ có trị riêng là $\lambda$ khi tồn tại một vector $\mathbf{x} \in \mathbb{R}^{n}$ thỏa mãn:
$$\mathbf{Ax} = \lambda \mathbf{x}$$
Khai triển biểu thức trên để đưa về dạng tích của $\mathbf{x}$:
$$\begin{eqnarray}
\mathbf{Ax} - \lambda \mathbf{x} & = & 0\\
\mathbf{Ax} - \lambda \mathbf{I}_{n} \mathbf{x} & = & 0\\
(\mathbf{A} - \lambda \mathbf{I}_{n}) \mathbf{x} & = & 0 \tag{3}\\
\end{eqnarray}$$
Dòng thứ 2 đạt được là nhờ tích của một ma trận với ma trận đơn vị thì bằng chính nó ($\mathbf{I}_{n} $ là kí hiệu cho ma trận đơn vị bậc $n$). Chúng ta gọi $\mathbf{x}$ là vector riêng (eigen vector) ứng với trị riêng $\lambda$.

Trị riêng và vector riêng của ma trận có những tính chất đặc biệt sau:

1. Nếu $\mathbf{x}$ là vector riêng tương ứng của trị riêng $\lambda$ thì $k\mathbf{x}$ cũng là một vector riêng của $\lambda$. Tính chất này cho thấy một trị riêng có thể có nhiều vector riêng. Tuy nhiên với một vector riêng chỉ có duy nhất một trị riêng.
1. Một ma trận có $n$ trị riêng bao gồm cả lặp lại và trị riêng phức. Điều này có được là vì trị riêng  là nghiệm của một đa thức bậc $n$. Thật vậy, biểu thức (3) về bản chât là tổ hợp tuyến tính của các cột ma trận $\mathbf{A} - \lambda \mathbf{I}_{n}$ với các phần tử của vector $\mathbf{x}$. Do kết quả bằng 0 nên các cột của ma trận $\mathbf{A} - \lambda \mathbf{I}_{n}$ là phụ thuộc tuyến tính. Từ đó suy ra $\mathbf{det}(\mathbf{A} - \lambda \mathbf{I}_{n}) = 0$. Triển khai định thức này ta thu được một đa thức $\mathbf{P}_{n}(\lambda)$ bậc $n$. Do đó ma trận có $n$ trị riêng bao gồm cả lặp và phức.
2. Khi ma trận $\mathbf{A}$ là ma trận đối xứng thì các trị riêng của $\mathbf{A}$ là các số thực.
3. Khi ma trận $\mathbf{A}$ là ma trận [xác định dương](https://vi.wikipedia.org/wiki/Ma_trận_(toán_học) thì các trị riêng của nó là các số thực dương và nếu ma trận $\mathbf{A}$ là ma trận nửa xác định dương thì các trị riêng của nó không âm. Chứng minh như sau: $\lambda \mathbf{x^{T}x} = \mathbf{x^{T}Ax} > 0$ khi $\mathbf{A}$ xác định dương. Mặt khác $\lambda \mathbf{x^{T}x} = \lambda \mathbf{||x||_{2}^2}, \mathbf{||x||_{2}^2} > 0 \forall \mathbf{x} \neq 0$. Suy ra $\lambda > 0$, như vậy mọi trị riêng của $\mathbf{A}$ đều dương. Chứng minh tương tự cho trường hợp $\mathbf{A}$ bán xác định đương.
4. Tổng các phần tử trên đường chéo chính của ma trận $\mathbf{A} \in \mathbb{R}^{n \times n}$ thì bằng tổng các trị riêng. Để chứng minh công thức này cần sử dụng đến phép phân tích riêng sẽ được trình bày bên dưới. Khi ma trận $\mathbf{A}$ độc lập tuyến tính nó có thể biểu diễn dưới dạng phân tích riêng như sau:
$$\mathbf{A = PDP^{-1}}$$
Áp dụng hằng đẳng thức $\mathbf{trace(AB) = trace(BA)}$ ta có:
$$\mathbf{trace(A) = trace(PDP^{-1}) = trace((PD)P^{-1}) = trace(P^{-1}PD) = trace(D)}$$
Từ đó suy ra tổng các phần tử trên đường chéo chính của ma trận $\mathbf{A}$ bằng tổng các trị riêng.
5. Đinh thức của ma trận $\mathbf{A} \in \mathbb{R}^{n \times n}$ thì bằng tích các trị riêng của nó.
Sử dụng phép phân tích riêng đối với ma trận $\mathbf{A}$ độ lập tuyến tính ta có:
$$\mathbf{A = PDP^{-1}} \Rightarrow \mathbf{det(A) = det(PDP^{-1}) = det(P).det(D).det(P^{-1}) = det(D)}$$.

**Phép phân tích riêng:** Phép phân tích riêng (EigenDecomposition) cũng là một dạng matrix factorization. Nó có mối liên hệ chặt chẽ với SVD. Trong đại số tuyến tính chúng ta luôn có thể phân tích một ma trận vuông độc lập tuyến tính $\mathbf{A} \in \mathbb{R}^{n \times n}$ thành tích của những ma trận vuông $\mathbf{P} \in \mathbb{R}^{n \times n}$ khả nghịch và ma trận đường chéo $\mathbf{D} \in \mathbb{R}^{n \times n}$ theo công thức:
$$\mathbf{A} = \mathbf{PDP}^{-1}$$
Đẳng thức trên tương đương với:
$$\mathbf{AP} = \mathbf{PD}$$
Bây giờ ta chỉ xét đến cột thứ i của cả 2 ma trận bên vế trái và phải:
$$\mathbf{Ap_{i}} = \mathbf{Pd_{i}}$$
Trong đó $\mathbf{p_{i}, d_{i}}$ lần lượt là cột thứ i của ma trận $\mathbf{P}$ và $\mathbf{D}$. Mặt khác do $\mathbf{D}$ là ma trận đường chéo nên $\mathbf{d_{i}}$ chỉ có duy nhất một phần tử khác 0 là $d_{ii}$ nên $\mathbf{Pd_{i}} = \mathbf{p_{i}}d_{ii}$. Như vậy:
$$\mathbf{Ap_{i}} = \mathbf{p_{i}}d_{ii}$$
Đây chính là phương trình xác định trị riêng của $\mathbf{A}$. Ta có thể thấy $d_{ii}$ chính là trị riêng của ma trận $\mathbf{A}$ và $\mathbf{p_{i}}$ là các vector riêng tương ứng của $d_{ii}$.

Như vậy điểm đặc biệt của phân tích riêng đó là đường chéo chính của $\mathbf{D}$ là các trị riêng của ma trận $\mathbf{A}$ và các cột của $\mathbf{P}$ là các vector riêng tương ứng với trị riêng nằm trên đường chéo chính. Ngoài ra phép phân tích riêng không là duy nhất. Nếu ma trận trực giao $\mathbf{P}$ thỏa mãn phương trình phân tích riêng thì ma trận $k\mathbf{P}$ cũng thỏa mãn phương trình phân tích riêng đó.


## 1.3. Phát biểu phân tích suy biến.
Trong đại số tuyến tính, phép phân tích suy biến là một dạng của `matrix factorization` nhằm phân tích một ma trận thành tích của các ma trận số thực hoặc ma trận số phức. Đây là một tổng quát tổng quát của phân tích riêng trong trường hợp ma trận không vuông khi nó biểu diễn ma trận gốc thành tích của  hai ma trận trực giao và một ma trận đường chéo. Phát biểu của phép phân tích suy biến sẽ được trình bày có dạng như sau.

### 1.3.1. Phân tích suy biến. 
Để đơn giản ta sẽ kí hiệu chỉ số kích thước ma trận ở bên dưới. Ma trận $\mathbf{A}_{mn}$ là kí hiệu cho ma trận $\mathbf{A} \in \mathbb{R}^{m \times n}$. Phép phân tích suy biến của ma trận $\mathbf{A}_{mn}$ là một dạng matrix factorization. Cũng giống như phương pháp phân tích riêng, nó sẽ biểu diễn ma trận gốc dưới dạng tích của 3 ma trận đặc biệt như sau:
$$\mathbf{A}_{mn} = \mathbf{U}_{mm}\mathbf{\Sigma}_{mn}\mathbf{V}_{nn}^\mathbf{T}$$
Trong đó $\mathbf{U}_{mm}, \mathbf{V}_{nn}$ là các ma trận trực giao và $\mathbf{\Sigma}_{mn}$ là ma trận đường chéo. 

Về mặt hình học phép phân tích suy biến sẽ lần lượt trải qua: phép xoay (rotation), phép nới rộng (scaling) và phép xoay ở cuối cùng. Nếu ta coi các dòng của ma trận $\mathbf{A}$ là các điểm dữ liệu và các chiều dữ liệu là các cột. Khi đó nhân các điểm dữ liệu với ma trận trực giao $\mathbf{U}_{mm}$ chính là việc ta thực hiện một phép xoay và phép xoay này không làm thay đổi tích vô hướng của các vector. Phép nhân với ma trận đường chéo $\mathbf{\Sigma}$ sẽ co dãn độ lớn các chiều theo giá trị của các trị riêng trên đường chéo chính. Và cuối cùng phép nhân với ma trận trực giao $\mathbf{V^{T}}$ lại thực hiện phép biến xoay một lần nữa. 

![](https://raw.githubusercontent.com/phamdinhkhanh/MovieLen/master/svd2.png)

> Hình 1: Minh họa biến đổi hình học của phép phân tích suy biến. (Nguồn [wikipedia](https://en.wikipedia.org/wiki/Singular_value_decomposition#/media/File:Singular-Value-Decomposition.svg))

### 1.3.2. Mối liên hệ giữa phân tích suy biến và phân tích riêng.

Ta có thể thấy phân tích suy biến khác với phân tích riêng ở chỗ nó áp dụng cho ma trận bất kì mà không yêu cầu ma trận đó phải vuông. Để thấy được mối liên hệ mật thiết giữa phép phân tích suy biến và phân tích riêng sau khi khai triển tích $\mathbf{A^{T}A}$:

$$\begin{eqnarray}\mathbf{A^{T}A} & = & \mathbf{(U\Sigma V^{T})^{T}}\mathbf{U\Sigma V^{T}} \\
& = & \mathbf{V \Sigma^{T}U^{T}}\mathbf{U\Sigma V^{T}} \\
& = & \mathbf{V \Sigma^{T}}\mathbf{\Sigma V^{T}}
\end{eqnarray}$$

Dấu bằng thứ 3 xảy ra là vì $\mathbf{U^{T}U} = \mathbf{I_n}$. Ngoài ra $\mathbf{\Sigma^{T}\Sigma}$ là một ma trận đường chéo có các thành phần trên đường chéo chính lần lượt là $\sigma_1^{2}, \sigma_2^{2},...,\sigma_{n}^2 > 0$. Như vậy $\mathbf{A^{T}A}$ là một phân tích riêng của ma trận trực giao $\mathbf{V}$ và ma trận đường chéo $\mathbf{\Sigma^{T}\Sigma}$. Hoàn toàn tương tự ta cũng có $\mathbf{AA^{T}}$ là một phân tích riêng cua ma trận trực giao $\mathbf{U}$ và ma trận đường chéo $\mathbf{\Sigma\Sigma^{T}}$. Các phần tử $\sigma_i$ trên đường chéo chính của $\mathbf{\Sigma}$ được gọi là giá trị suy biến (singular values) của ma trận $\mathbf{A}$. Các cột của ma trận $\mathbf{V}$ là hệ vector riêng ứng với trị riêng của $\mathbf{\Sigma^{T}\Sigma}$ và còn được gọi là hệ vector suy biến trái (left-singular vectors). Các cột của $\mathbf{U}$ được gọi là hệ vector suy biến phải (right-singular vectors).

### 1.3.3. Phân tích suy biến trong python
Trong python phép phân tích suy biến của một ma trận được thực hiện dễ dàng thông qua việc sử dụng hàm *SVD* của package scipy như sau:

In [ ]:
import scipy.linalg as ln
import numpy as np
m, n = 2, 3
n_diag = min(m, n)
#Init normal standard random variable A with size (m, n)
A = np.random.rand(m, n)
U, S_diag, V = ln.svd(A)
#Create diagonal matrix S based on diagonal
S = np.zeros((n_diag, n_diag))
np.fill_diagonal(S, S_diag)
if m > n:
    S = np.concatenate((S, np.zeros((1, n))), axis = 0)
elif m < n:
    S = np.concatenate((S, np.zeros((m, 1))), axis = 1)
        
print('Matrix A: \n %s \n'%A)
print('orthogonal matrix U: \n %s \n'%U)
print('Check Frobenius U^TU-I: \n %s \n'%ln.norm(np.dot(U.T,U)-np.eye(m, m), 'fro'))
print('orthogonal matrix V: \n %s \n'%V)
print('Check Frobenius V^TV-I: \n %s \n'%ln.norm(np.dot(V.T,V)-np.eye(n, n), 'fro'))
print('Diagonal matrix S: \n %s \n'%S_diag)
print('Matrix S: \n %s \n'%S)
print('Check Frobenius U.S.V - A: \n %s \n'%ln.norm(np.dot(U, S.dot(V))-A,'fro'))

## 1.4. Các phép giảm chiều SVD.
Thông thường việc phân tích suy biến một ma trận có kích thước lớn sẽ rất lâu vì đòi hỏi phải giải phương trình đặc trưng để tìm ra các giá trị đặc trưng, từ đó suy ra ma trận đường chéo $\mathbf{\Sigma}$. Từ phương trình phân tích riêng $\mathbf{A^{T}A} = \mathbf{V\Sigma^{T}\Sigma V^{T}}$ suy ra $\mathbf{A^{T}AV} = \mathbf{V\Sigma^{T}\Sigma}$, sử dụng phương trình ứng với từng cột cả 2 vế trái và phải để tính ra được các vector riêng ứng với mỗi trị riêng và suy ra được ma trận $\mathbf{V}$. Cách tìm ma trận $\mathbf{U}$ cũng được suy ra tương tự từ phương trình phân tích riêng $\mathbf{AA^{T}} = \mathbf{U\Sigma\Sigma^{T} U^{T}}$. Quá trình này phải trải qua nhiều bước và khi kích thước ma trận lớn, chi phí thời gian và lưu trữ sẽ rất lớn. Vì vậy các dạng giảm chiều SVD sẽ rút gọn quá trình tính toán.


### 1.4.1. Phương pháp làm mỏng SVD.
Xuất phát từ ý tưởng số quan sát thường lớn gấp rất nhiều lần so với số chiều hay $m >> n$ trong hầu hết các trường hợp của ma trận $\mathbf{A}$ nên thay vì phải tính toán bộ ma trận $\mathbf{U}_{mm}$ ta sẽ chỉ tính n cột đầu tiên là $\mathbf{U}_{mn}$. Số chiều của ma trận đường chéo $\mathbf{\Sigma}_{mn}$ cũng giảm xuống thành $\mathbf{\Sigma}_{nn}$ . Khi đó ma trận $\mathbf{A}$ được biểu diễn dưới dạng:
$$\mathbf{A} = \mathbf{U}_{mn}\mathbf{\Sigma}_{nn}\mathbf{V}_{nn}^\mathbf{T}$$
Như vậy số lượng các trị riêng cần tìm chỉ còn $n$ và số lượng vector riêng chỉ còn $2n$ ($n$ cột của ma trận $\mathbf{U}_{mn}$ và $n$ cột của ma trận $\mathbf{V}_{nn}$).

### 1.4.2 . Phương pháp Compact SVD.
Ta có thể biểu diễn ma trận $\mathbf{A}$ dưới dạng tổng của các tích vector cột $\mathbf{u_i} \in \mathbb{R}^{m}$ của $\mathbf{U}_{mm}$ và vector dòng $\mathbf{v_i} \in \mathbb{R}^{n}$ của $\mathbf{V}_{nn}^{\mathbf{T}}$ như sau:
$$\mathbf{A} = \sum_{i = 1}^{n}\mathbf{u_i\sigma_i v_i}\tag{4}$$
Các vector $\mathbf{u_i}$ và $\mathbf{v_i}$ là các hệ cơ sở độc lập tuyến tính. Thông thường trong ma trận đường chéo $\mathbf{\Sigma}_{nn}$ chỉ một lượng lớn các trị riêng có lớn hơn 0. Các trị riêng còn lại đều xấp xỉ 0. Do đó chỉ tại $r$ vị trí dòng và cột tương ứng với các trị riêng đủ lớn ta mới thực hiện tính toán SVD. Biểu diễn ma trận $\mathbf{A}_{mn}$ dưới dạng compact SVD như sau:
$$\mathbf{A} = \mathbf{U}_{r}\mathbf{\Sigma}_{r}\mathbf{V}_{r}^\mathbf{T}$$
Trong đó các ma trận $\mathbf{U_r, \Sigma_r, V_r}^{\mathbf{T}}$ lần lượt là các ma trận sau khi đã rút gọn các dòng và cột để chỉ giữ lại các vị trí tương ứng với $\sigma_i$ đủ lớn.
Nếu $r << n$ thì cách tính này tiết kiệm được nhiều số lượng tính toán và lưu trữ hơn so với phương pháp làm mỏng SVD.

### 1.4.3. Phương pháp Truncate SVD.
Trong phương pháp này ta giả định ma trận $\mathbf{A}$ là ma trận bán xác định dương. Khi đó mọi trị riêng của nó đều không âm. Phương pháp Truncate SVD cũng tương tự như Compact SVD. Tuy nhiên thay vì các dòng và cột tương ứng với trị riêng đủ lớn trên ma trận đường chéo $\mathbf{\Sigma}_{mn}$ thì chúng ta sẽ chỉ lấy ra $t$ dòng và cột ứng với top $t$ các trị riêng $\sigma_1 > \sigma_2 >...> \sigma_t > 0$ lớn nhất của $\mathbf{A}$ từ $\mathbf{U, V}^{\mathbf{T}}$ . Phần còn lại của ma trận sẽ bị loại bỏ. Như vậy trong phương pháp Truncate SVD ta sẽ thu được ma trận xấp xỉ của ma trận $\mathbf{A}$ là ma trận:
$$\mathbf{\hat{A}} = \mathbf{U}_{t}\mathbf{\Sigma}_{t}\mathbf{V}_{t}^\mathbf{T}$$
Hoặc ta có thể biểu diễn dưới dạng tổng của tích vô hướng các vector cột và dòng của $\mathbf{U, V}^{\mathbf{T}}$ như sau:
$$\mathbf{A} = \sum_{i = 1}^{t}\mathbf{u_i\sigma_i v_i}\tag{5}$$
Khi đó ta còn tính được khoảng cách norm Frobenius giữa $\mathbf{\hat{A}}$ và $\mathbf{A}$ chính bằng tổng bình phương của các trị riêng còn lại từ $\sigma_{t+1}$ đến $\sigma_n$ (với giả định $\mathbf{A}$ có $n$ trị riêng) như sau:

$$\begin{eqnarray}||\mathbf{A}-\mathbf{\hat{A}}||_{F}^{2} & = & (\mathbf{A}-\mathbf{\hat{A}})^{\mathbf{T}}(\mathbf{A}-\mathbf{\hat{A}}) \\
& = & (\sum_{i = t+1}^{n}\mathbf{u_i\sigma_i v_i})^{\mathbf{T}}(\sum_{i = t+1}^{n}\mathbf{u_i\sigma_i v_i}) \\
& = & (\sum_{i = t+1}^{n}\mathbf{\sigma_i v_i^{T} u_i^{T}})(\sum_{i = t+1}^{n}\mathbf{\sigma_i u_i v_i}) \\
& = & \sum_{i = t+1}^{n}\sum_{j = t+1}^{n}\mathbf{\sigma_i \sigma_j v_i^{T} u_i^{T}u_j v_j} \\
& = & \sum_{i = t+1}^{n}\mathbf{\sigma_i^{2} v_i^{T} u_i^{T}}\mathbf{u_i v_i} \\
& = & \sum_{i = t+1}^{n}\mathbf{\sigma_i^{2} v_i^{T} v_i} \\
& = & \sum_{i = t+1}^{n}\mathbf{\sigma_i^{2}} \\
\end{eqnarray}$$

Dấu bằng thứ 3 là vì lấy đẳng thức (4) trừ đi (5). Dấu bằng thứ 5 xảy ra là do ma các vector cột của $\mathbf{U, V}$ là những hệ trực giao nên $\mathbf{u_i^{T}u_j} = 0, \mathbf{v_i^{T}v_j} = 0 , ~ \forall 1 \leq i \neq j \leq n$. Các dấu bằng 6, 7 là do $\mathbf{u_i^{T}u_i} = 1, \mathbf{v_i^{T}v_i} = 1,~ \forall 1 \leq i \leq n$. Như vậy ta đã hiểu lý do tại sao chúng ta chỉ chọn ra top $t$ trị riêng có giá trị lớn nhất. Khi đó sai số của 2 ma trận sẽ là nhỏ nhất vì bằng tổng bình phương của các trị riêng còn lại. Phương pháp truncate SVD còn cho ta biết được tỷ lệ phần trăm lượng thông tin lưu giữ trong ma trận xấp xỉ thông qua công thức:
$$\frac{\sum_{i = 1}^{t}\sigma_i^2}{\sum_{j = 1}^{n}\sigma_j^2}$$

## 1.5. Bài toán xấp xỉ low-rank.
Bài toán xấp xỉ low-rank là một trong những ứng dụng phổ biến của phân tích suy biến. Mục tiêu chính của bài toán là tìm một ma trận xấp xỉ tốt nhất với ma trận gốc nhưng có hạng xác định trước và nhỏ hơn hạng của ma trận gốc. Mức độ xấp xỉ được đo lường thông qua chuẩn Frobenius của hiệu 2 ma trận. Bài toán có dạng như sau:
$$\arg\min_{rank(X) = k} ||\mathbf{\mathbf{A-X}||_{F}^{2}}$$
Người ta chứng minh được rằng nghiệm của bài toán low-rank chính là phép truncate SVD đối với $k$ trị riêng lớn nhất. Xem chứng minh [SVD - princeton](https://www.cs.princeton.edu/courses/archive/spring12/cos598C/svdchapter.pdf). 


# 2. Áp dụng phân tích suy biến
## 2.1. Giải phương trình tuyên tính.
SVD được sử dụng trong việc giải các phương trình hồi qui tuyến tính trong trường hợp tổng quát. Như chung ta đã biết bài toán hồi qui tuyến tính sẽ tìm một phương trình có dạng tuyến tính sao cho khác biệt giữa các điểm dữ liệu dự báo và thực tế là nhỏ nhất. Khác biệt này được đo lường thông qua một hàm loss function có dạng:
$$\mathcal{L}(\mathbf{w}) = \frac{1}{2n}||\mathbf{y - Xw}||_2^{2}$$
Trong đó $\mathbf{y} \in \mathbb{R}^{m}$ là vector biến mục tiêu (hoặc biến phụ thuộc). Ma trận $\mathbf{X} \in \mathbb{R}^{m \times n}$ là ma trận mở rộng của ma trận đầu vào có mỗi dòng là một quan sát và mỗi cột là một chiều dữ liệu. Ma trận này có cột cuối cùng bằng 1 đại diện cho hệ số tự do. $\mathbf{w} \in \mathbb{R}^{n}$ là vector các hệ số của phương trình hồi qui và là đại lượng biến đổi trong phương trình loss function.
Thông qua việc giải phương trình đạo hàm bậc nhất của hàm loss function ta có thể tìm được $\mathbf{w}$.

Theo phương trình đạo hàm tích (Product rules) ta có: $\nabla_\mathbf{x} (f^{\mathbf{T}}g) = (\nabla_\mathbf{x}f)g + (\nabla_\mathbf{x}g) f$.
Do đó đạo hàm $\nabla_\mathbf{x} (f^{\mathbf{T}}f) = 2(\nabla_\mathbf{x}f)f$. Áp dụng vào triển khai đạo hàm $\mathcal{L}(\mathbf{w})$.

$$\begin{eqnarray}
\frac{\partial \mathcal{L}(\mathbf{w})}{\partial\mathbf{w}} & = & \frac{1}{2n}\frac{\partial (\mathbf{y - Xw})^{\mathbf{T}}(\mathbf{y - Xw})}{\partial\mathbf{w}} \\
& = & \frac{1}{n}\frac{(\partial(\mathbf{y - Xw}))(\mathbf{y - Xw})}{\partial\mathbf{w}} \\
& = & \frac{1}{n}\frac{\mathbf{X^{T}(y - Xw)}}{\partial\mathbf{w}}
\end{eqnarray}$$

Đẳng thức dòng thứ 3 xảy ra là do $\nabla_{\mathbf{x}} \mathbf{Xw} = \mathbf{X^T}$. 
Đặt $\mathbf{A} := \mathbf{X^TX}$ và $\mathbf{X^Ty} := \mathbf{b}$ 
Như vậy nghiệm $\mathbf{w}$ của hàm loss function sẽ thỏa màn $\mathbf{Aw} = \mathbf{b}$. Trong trường hợp $\mathbf{A}$ khả nghịch phương trình có nghiệm $\mathbf{w} = \mathbf{A}^{-1}\mathbf{b}$. Tuy nhiên trường hợp $\mathbf{A}$ không khả nghịch để tìm nghiệm của bài toán ta cần sử dụng đến ma trận giả nghịch đảo (còn gọi là ma trận Pseudo - Inverse hoặc ma trận Moore - Penrose) của $\mathbf{A}$ được kí hiệu là $\mathbf{A}^{\dagger}$. Các định nghĩa và tính chất về ma trận giả nghịch đảo có thể tìm hiểu tại [Pseudo - Inverse](http://www.sci.utah.edu/~gerig/CS6640-F2012/Materials/pseudoinverse-cis61009sl10.pdf). Ma trận giả nghịch đảo có thể được giải nhờ phép phân tích suy biến của $\mathbf{A}$. Nếu ma trận $\mathbf{A}$ có biểu diễn phân tích suy biến dạng:
$$\mathbf{A} = \mathbf{U\Sigma V^{T}}$$

Khi đó ma trận giả nghịch đảo của $\mathbf{A}$ sẽ là:
$$\mathbf{A}^{\dagger} = \mathbf{V\Sigma^{-1} U^{T}}$$
Ta thấy ma trận giả nghịch đảo vẫn có tính chất như ma trận nghịch đảo đó là $\mathbf{A}^{\dagger}\mathbf{A} = \mathbf{I}$.
Bên dưới ta sẽ sử dụng SVD để giải phương trình hồi qui tuyến tính.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

n = 100
x = np.arange(n)

# add 1 column in the last of X to create X matrix
X = np.concatenate((x.reshape(n, 1), np.ones((n, 1))), axis = 1)
y = np.random.randn(n)*10 + 2*x
plt.plot(x, y)

In [ ]:
# Calculate matrix A, b
A = np.dot(X.T, X)
b = np.dot(X.T, y)

# Matrix factorization SVD 
U, S_diag, V = np.linalg.svd(A)
S = np.zeros((S_diag.shape[0], S_diag.shape[0]))
np.fill_diagonal(S, S_diag)
S_inv = np.linalg.inv(S)

# Pseudo - inverse matrix
A_pse =  np.dot(V.T, np.dot(S_inv, U))

# Estimate coefficient
w_svd = np.dot(A_pse, b)
print('w calculated from SVD: %s' %str(w_svd))

In [ ]:
from sklearn import linear_model
ln_reg = linear_model.LinearRegression(fit_intercept = False)
ln_reg.fit(X, y)
print('w calculated from sklearn: %s' %str(ln_reg.coef_))

## 2.2. Giảm chiều dữ liệu hình ảnh.

Giả sử ta có một tập dữ liệu nhiều ảnh có kích thước rất lớn. Khả năng lưu trữ của server là có hạn. Trong tình huống này làm thể nào để giảm kích thước của bộ ảnh vừa với dung lượng server mà thông tin của các bức ảnh vẫn giữ được một lượng lớn. Câu trả lời sẽ có sau khi khi ta thực hành nén một bức ảnh bằng thuật toán truncate SVD.

**Bức ảnh gốc**

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.linalg as ln
from PIL import Image
import urllib.request
from io import BytesIO

%matplotlib inline

url = str('https://anh.eva.vn/upload/3-2017/images/2017-07-11/ghen-ti-voi-nhung-thu-chi-thuoc-ve-nu-than-han-quoc-song-hye-kyo-3-1499791578-width500height330.jpg')
with urllib.request.urlopen(url) as url:
    f = BytesIO(url.read())

ig = np.array(Image.open(f))
print('Image shape: %s'%str(ig.shape))
# Convert to grey
ig = ig.dot([0.299, 0.5870, 0.114])
plt.imshow(ig)

Ta sẽ thực hiện nén ảnh theo phương pháp SVD sao cho chỉ lấy lần lượt với 10 và 20 trị riêng lớn nhất để lưu giữ thông tin.

In [ ]:
def SVD_pic(n_evl):
    """
    n_evl: number of highest eigenvalues taken
    """
    #Take m, n shape
    m = ig.shape[0]
    n = ig.shape[1]
    #Singular Value Decomposition
    U, S, V = ln.svd(ig) 
    #Get id position of n highest eigenvalues
    id_trunc = np.argsort(S)[::-1][:n_evl]
    #Extract matrix U_t, V_t, S_t
    U_t = U[np.ix_(np.arange(m), id_trunc)]
    V_t = V[np.ix_(id_trunc, np.arange(n))]
    S_diag = S[id_trunc]
    S_t = np.zeros((n_evl, n_evl))
    np.fill_diagonal(S_t, S_diag)
    #Return picture
    A = np.dot(U_t, S_t.dot(V_t))
    #Norm Frobenius
    fb = ln.norm(A-ig, 'fro')
    prt_retain = (1-fb**2/np.sum(S**2))*100
    plt.imshow(A)
    print('Percentage of information retained: %.2f%s \n'%(prt_retain, '%'))

**Kết quả nén ảnh với lần lượt 10 và 20 trị riêng lớn nhất**

In [ ]:
SVD_pic(10)

In [ ]:
SVD_pic(20)

Như vậy với 10 trị riêng lớn nhất đã thể hiện được được 99.39% thông tin bức ảnh và với 20 trị riêng lớn nhất thể hiện được 99.74% thông tin bức ảnh. So với bức ảnh gốc ta phải lưu trữ ma trận có kích thước là 330x500. Trong khi với 20 chiều vector ta chỉ phải lưu trữ các ma trận $\mathbf{U_t}, \mathbf{V_t}$ có kích thước là 330x20, 20x500 và ma trận $\mathbf{S}$ có kích thước là 20x20. Giả định mỗi phần tử của ma trận được lưu với cùng số byte. Như vậy số lượng thông tin ta cần lưu chỉ là:
$$\frac{330 \times 20 + 20 \times 500 + 20 \times 20}{330 \times 500} = 10 \text{%}$$
Dó đó phương pháp truncate SVD thường được sử dụng trong nén thông tin rất hiệu quả.

## 2.2. Áp dụng cho thuật toán recommendation system.

Trong các thuật toán recommendation phương pháp SVD cũng tỏ ra hiệu quả hơn so với các phương pháp `matrix factorization` sử dụng `gradient descent`. Ý tưởng khi áp dụng vào recommendation system cũng tương tự như nén ảnh đó là căn cứ vào những cặp (user, item) đã được rating của ma trận tiện ích ta sẽ tìm ra một ma trận xấp xỉ tốt nhất với ma trận tiện ích. Sử dụng ma trận xấp xỉ để dự báo cho những cặp (user, item) chưa được rating.

In [ ]:
class Data(object):
    """
    This class used to manage data.
    Two arguments:
    dataset: pandas data frame include user_id, item_id and rating
    split_rate: number train ratings/ total ratings
    """
    def __init__(self, dataset, split_rate):
        self.dataset = dataset
        self.split_rate = split_rate
        self.train, self.test = self.split_train_test(self.dataset)
        self.n_users = np.max(self.train[:, 0] + 1) #plus one because index start from 0
        self.n_items = np.max(self.train[:, 1] + 1)
        self.Ytrain, self.Rtrain = self.utility_matrix(self.train)
        self.Ytest , self.Rtest  = self.utility_matrix(self.test)
        self.Ystad,  self.u_mean = self.standardize_Y(self.Ytrain)
        self.n_ratings = self.train.shape[0]
        
    def split_train_test(self, dataset):
        "split train and test"
        gb = dataset.groupby('user_id')
        ls = [gb.get_group(x) for x in gb.groups]
        items = [x for x in gb.groups]
        index_size = [{'i': i, 'index':gb.groups[i], 'size':len(gb.groups[i])} for i in items]
        index_train = pd.Int64Index([])
        index_test = pd.Int64Index([])
        for x in index_size:
            np.random.shuffle(x['index'].values)
            le = int(x['size']*self.split_rate)
            index_train = index_train.append(x['index'][:le])
            index_test = index_test.append(x['index'][le:])
        train = dataset.iloc[index_train].values
        test = dataset.iloc[index_test].values
        #minus id to 1 to index start from 0
        train[:, 0] -= 1
        train[:, 1] -= 1
        test[:, 0] -= 1
        test[:, 1] -= 1
        return train, test
    
    def utility_matrix(self, data_mtx):
        "create Y and R matrix"
        Y = np.zeros(shape = (self.n_items, self.n_users))
        Y = sparse.coo_matrix((data_mtx[:, 2], (data_mtx[:, 1], data_mtx[:, 0])), \
                              shape = (self.n_items, self.n_users), dtype = np.float).toarray()
        R = sparse.coo_matrix((np.ones((data_mtx.shape[0],)), (data_mtx[:, 1], data_mtx[:, 0])), \
                              shape = (self.n_items, self.n_users)).toarray()
        return Y, R
    
    def standardize_Y(self, Y):
        "standard data to mean ratings of each user = 0"
        sum_rating = Y.sum(axis = 0)
        u_rating = np.count_nonzero(Y, axis = 0)
        u_mean = sum_rating/u_rating
        for n in range(self.n_users):
            for m in range(self.n_items):
                if Y[m, n] != 0:
                    Y[m, n] -= u_mean[n]
        return Y, u_mean

In [ ]:
class Model():
    """
    This class manage update U and I matrix, predict and evaluate error
    Two arguments:
    data: instance from Data class which supplies the data for model
    n_evl: number of eigenvalues
    """
    def __init__(self, data, n_evl = None):
        self.data = data
        self.n_evl = n_evl
        self.Yhat_std = self.fit(self.data, self.n_evl)
        self.Yhat = self.pred()
        
    def fit(self, data, n_evl):
        U, S, V = ln.svd(data.Ystad)
        #When n_evl is undefined, we set it equal to diagonal length
        if self.n_evl is None:
            n_evl = S.shape[0]
        #Get id position of n highest eigenvalues
        id_trunc = np.argsort(S)[::-1][:n_evl]
        #Extract matrix U_t, V_t, S_t
        U_t = U[np.ix_(np.arange(U.shape[0]), id_trunc)]
        V_t = V[np.ix_(id_trunc, np.arange(V.shape[1]))]
        S_diag = S[id_trunc]
        S_t = np.zeros((n_evl, n_evl))
        np.fill_diagonal(S_t, S_diag)
        #Return matrix forecast
        self.Yhat_std = np.dot(U_t, S_t.dot(V_t))
        return self.Yhat_std
    
    def pred(self):
        #invert to forecast values by plus user's mean ratings
        self.Yhat = self.Yhat_std.copy()
        for n in range(self.data.n_users):
            self.Yhat[:, n] += self.data.u_mean[n]
        #convert to interger values because of rating is integer
        self.Yhat = self.Yhat.astype(np.int32) 
        #replace values > 5 by 5 and values < 1 by 1
        self.Yhat[self.Yhat > 5] = 5
        self.Yhat[self.Yhat < 1] = 1
        return self.Yhat

    def pred_train_test(self, Yhat, R):
        #replace values have not yet rated by 0 
        Y_pred = Yhat.copy()
        Y_pred[R == 0] = 0
        return Y_pred
    
    def RMSE(self, Y, Yhat):
        error = Y - Yhat
        n_ratings = np.sum(Y != 0)
        rmse = math.sqrt(np.linalg.norm(error, 'fro')**2/n_ratings)
        return rmse

In [ ]:
class MF():
    """
    This class used to manage model and data
    Two main arguments:
    data: control the data
    model: control the functions which execute model
    """
    def __init__(self, data, model):
        self.data = data
        self.model = model
        self.Y_pred_train = None
        self.Y_pred_test = None
        self.Yhat = None
        
    def fit(self):
        #fiting model
        self.model.fit(self.data, self.model.n_evl)
        #calculate Y_hat
        self.Yhat = self.model.pred()
        #calculate Y_pred_train by replace non ratings by 0
        self.Y_pred_train = self.model.pred_train_test(self.Yhat, self.data.Rtrain)
        self.Y_pred_test  = self.model.pred_train_test(self.Yhat, self.data.Rtest)
        print('RMSE: {}'.\
              format(self.model.RMSE(self.data.Ytest, self.Y_pred_test)))

    def recommend_for_user(self, user_id, k_neighbors):
        recm = np.concatenate((np.arange(1, self.Y_pred_test.shape[0]+1).reshape(-1, 1), \
                               self.Y_pred_test[:, user_id - 1].reshape(-1, 1)), axis = 1)
        recm.sort(axis = 0)
        print('Top %s item_id recommended to user_id %s: %s'%\
              (k_neighbors, user_id, str(recm[-k_neighbors:, 0])))

In [ ]:
import numpy as np
import scipy.linalg as ln
import pandas as pd
import scipy.sparse as sparse
import math

columns = ['user_id', 'item_id', 'rating', 'timestamp']
movie_length = pd.read_csv('../input/ratings.dat', header = 0, \
                           names = columns, sep = '::', engine = 'python')
movie_length = movie_length.sort_values(['user_id', 'item_id'])

data = Data(dataset = movie_length, split_rate = 2/3)

In [ ]:
# Fitting model  
model = Model(data = data, n_evl = 2)
mf = MF(data = data, model = model)
mf.fit()

## 2.3. Tài liệu tham khảo.
1. [Các định nghĩa liên quan đến ma trận - Wiki](https://vi.wikipedia.org/wiki/Ma_tr%E1%BA%ADn_(to%C3%A1n_h%E1%BB%8Dc))
2. [Singular Value Decomposition - Blog machine learning cơ bản](https://machinelearningcoban.com/2017/06/07/svd/)
3. [Singular Value Decomposition - Princeton](https://www.cs.princeton.edu/courses/archive/spring12/cos598C/svdchapter.pdf)
4. [Singular Value Decomposition - Standford - Youtube](https://www.youtube.com/watch?v=P5mlg91as1c)
5. [Singular Value DecomposItion - wiki](https://en.wikipedia.org/wiki/Singular_value_decomposition)
6. [Singular Value DecomposItion - Carnegie Mellon University](https://www.cs.cmu.edu/~venkatg/teaching/CStheory-infoage/book-chapter-4.pdf)
7. [Singular Value DecomposItion - Standford - Stephen Boyd](https://see.stanford.edu/materials/lsoeldsee263/16-svd.pdf)
8. [Pseudo - Inverse, Least Squares, PCA, SVD - Utah university](http://www.sci.utah.edu/~gerig/CS6640-F2012/Materials/pseudoinverse-cis61009sl10.pdf)